In [24]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import time
from datetime import datetime

current_time = time.time()
current_date = datetime.utcfromtimestamp(current_time).strftime('%Y-%m-%d')

# print("Current date:", current_date)


openai_api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(
    temperature=0.2,
    model_name="gpt-3.5-turbo",
    openai_api_key=openai_api_key,
    max_tokens=1000,
)
# Create a single memory instance for the entire conversation
memory = ConversationBufferMemory(return_messages=True)


user = "Hassan"
gender = "male"
designation = "employee"
salary = "Rs. 50,000"
job_status = "permanent"
end_date = "present"

petrol_claim = "25 Liter per month"
mobile_claim = "Rs 500 per month"
house_finance = "upto 1 million with 6% interest per annum"
auto_finance = "upto 1 million with 5% interest per annum"
insurance_type = "Basic"
health_insurance_details = "Coverage up to 4K units with 400 deductible."
accidental_death_insurance_details = "coverage of 10K units."


# user = "Hassan"
# gender = "male"
# designation = "employee"
# job_status = "probation"
joining_date = "2000-March-22"
# end_date = "2024-June-22"

# petrol_claim = "-"
# mobile_claim = "-"
# house_finance = "-"
# auto_finance = "-"
# insurance_type = "-"
# health_insurance_details = "-"
# accidental_death_insurance_details = "-"
	
current_date = datetime.utcfromtimestamp(current_time).strftime('%Y-%m-%d')

def generate_response(input_message, model, memory):
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
         You are Hira, a friendly female HR of Meezan Bank of Pakistan. Your role is to evaluate the pension based on their information.
        The current date is {current_date}

        {user} Profile:
            - Gender: {gender}.
            - Designation: {designation}.
            - Job Status: {job_status}.
            - Joining Date: {joining_date}.
            - Current Salary: {salary}.


        In this part, you will inform the {user} about their eligibility for pension based on their job status and the salary.

        The {user} job status is {job_status}. After checking the job status proceed with following point:
        - If {user}'s job status is permanent, inform them that you are qualified for the pension.
            1- Here is how you calculate the pension.
                1.1- You already know the joining date of the user now check the current date and evaluate that user has spend 20 years or not in the bank.
                1.2- You have already know the user information so you have to evaluate the pension based on their job status and the salary.
                1.3- If {user} already spent 20 years in the company then calculate the pension according to the salary and inform him/her that its not an actual pension \n
                    its just an estimated value. You have to verify this from the HR in person.
        - If {user}'s job status is probation then inform him that your probationary period will be 90 days and after your probationary period you have to spent atleast 20 years in the company to get pension.
        
        - You have to inform the employee who is on probation period that you are not eligible for the pension.


        
        You are strictly not allowed to talk on off topic.
        Your response should be precise and to the point.
        Stick to this professional tone.  
        Make sure you have asked all the questions that mentioned above before proceeding further queries.
        """),
        MessagesPlaceholder(variable_name="history"),
        ("human", f"{input_message}"),
    ])

    chain = (
        RunnablePassthrough.assign(
            history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
        )
        | prompt
        | model
    )

    inputs = {"input": input_message}
    response = chain.invoke(inputs)

    # Save the context for future interactions
    memory.save_context(inputs, {"output": response.content})
    memory.load_memory_variables({})

    return response.content

In [25]:
# Prompt 1
prompt = "Hi"
print(generate_response(prompt, chat, memory))

Hello, how can I assist you today?


In [26]:
# Prompt 1
prompt = "Who am I?"
print(generate_response(prompt, chat, memory))

You are Hassan.


In [27]:
# Prompt 1
prompt = "What should be my pension if I got retired."
print(generate_response(prompt, chat, memory))

As per your job status being permanent, you are qualified for the pension. To calculate the pension, we need to verify if you have spent 20 years in the company. If you have, then we can estimate the pension based on your salary.


In [28]:
# Prompt 1
prompt = "yes"
print(generate_response(prompt, chat, memory))

Great! Since you have confirmed that you have spent 20 years in the company, we can estimate your pension based on your salary. Your current salary is Rs. 50,000. I will calculate the estimated pension for you. Please wait for a moment.


In [29]:
# Prompt 1
prompt = "Ok"
print(generate_response(prompt, chat, memory))

Based on your salary of Rs. 50,000 and 20 years of service, the estimated pension amount is Rs. 25,000. Please note that this is just an estimated value. For the actual pension amount, you will need to verify with the HR department in person.
